## COLOMBIA COMPLETE COVID-19 DATASET

**Author: Camilo Esteban Ruiz**

Email: ruiznho123@gmail.com

[LinkedIn Profile](http://linkedin.com/in/camesruiz)

## Introduction

Version 3.0 (March 28th): Includes number of cases in ICU, added quarantine start marker on graphs to try to visualize effect on number of cases after the national quaratine was ordered by the government on March 24th.

Coronavirus (COVID-19) made its outbreak in Colombia with the first confirmed in the contry on march 06, since then, number of confirmed cases has been increasing and deaths related to the virus are starting to have the first confirmed cases.

This notebook emphazises on giving some insights on the virus spread in Colombia.

Feel free to click on the "Code" button above each output to go deeper into the code for the graphs.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib.pyplot import plot
import seaborn as sn
%matplotlib inline 

import geopandas as gpd

import folium
from folium import Choropleth
from folium.plugins import HeatMap

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex3 import *

from sklearn.linear_model import LinearRegression
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/colombia-shape-files-by-departments/depto.dbf
/kaggle/input/colombia-shape-files-by-departments/depto.prj
/kaggle/input/colombia-shape-files-by-departments/depto.shp
/kaggle/input/colombia-shape-files-by-departments/depto.shx
/kaggle/input/colombia-covid19-complete-dataset/covid-19-colombia.csv
/kaggle/input/colombia-covid19-complete-dataset/covid-19-colombia-all.csv
/kaggle/input/colombia-covid19-complete-dataset/Casos1.csv
/kaggle/input/colombia-covid19-complete-dataset/colombia_departamentos.csv
/kaggle/input/colombia-covid19-complete-dataset/covid-19-colombia-deaths.csv
/kaggle/input/colombia-covid19-complete-dataset/covid-19-colombia-confirmed.csv


In [2]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [3]:
#Data import

colombia_df = pd.read_csv('../input/colombia-covid19-complete-dataset/covid-19-colombia-all.csv')
confirmed = pd.read_csv('../input/colombia-covid19-complete-dataset/covid-19-colombia-confirmed.csv')
deaths = pd.read_csv('../input/colombia-covid19-complete-dataset/covid-19-colombia-deaths.csv',encoding='ISO-8859-1')
col_df = pd.read_csv('../input/colombia-covid19-complete-dataset/covid-19-colombia.csv')
cases = pd.read_csv('../input/colombia-covid19-complete-dataset/Casos1.csv')

In [4]:
cases.columns = ["ID", "date", "city", "departamento", "state", "age", "sex", "type", "procedence"]
cols_ = cases.select_dtypes(include=[np.object]).columns
cases[cols_] = cases[cols_].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

cases.date = pd.to_datetime(cases.date, dayfirst=True)

cases

,ID,date,city,departamento,state,age,sex,type,procedence
0,1,2020-03-06,Bogota,Bogota D.C.,Recuperado,19,F,Importado,Italia
1,2,2020-03-09,Buga,Valle del Cauca,Recuperado,34,M,Importado,Espana
2,3,2020-03-09,Medellin,Antioquia,Recuperado,50,F,Importado,Espana
3,4,2020-03-11,Medellin,Antioquia,Recuperado,55,M,Relacionado,Colombia
4,5,2020-03-11,Medellin,Antioquia,Recuperado,25,M,Relacionado,Colombia
...,...,...,...,...,...,...,...,...,...
1156,1157,2020-04-02,Bogota,Bogota D.C.,Casa,28,M,Importado,Estados Unidos
1157,1158,2020-04-02,Bogota,Bogota D.C.,Casa,38,F,En estudio,Colombia
1158,1159,2020-04-02,Bogota,Bogota D.C.,Casa,3,M,En estudio,Colombia
1159,1160,2020-04-02,Bogota,Bogota D.C.,Casa,48,M,En estudio,Colombia


In [5]:
cases.groupby(['departamento']).count()

,ID,date,city,state,age,sex,type,procedence
departamento,,,,,,,,
Antioquia,127,127,127,127,127,127,127,127
Atlantico,4,4,4,4,4,4,4,4
Barranquilla D.E.,31,31,31,31,31,31,31,31
Bogota D.C.,542,542,542,542,542,542,542,542
Bolivar,3,3,3,3,3,3,3,3
Boyaca,6,6,6,6,6,6,6,6
Caldas,16,16,16,16,16,16,16,16
Cartagena D.T. y C,39,39,39,39,39,39,39,39
Casanare,2,2,2,2,2,2,2,2


In [6]:
#Dataframe overview
col_df.head()

,date,confirmed,confirmed_daily,deaths,deaths_daily,recovered,recovered_daily
0,2020-03-05,0,0,0,0,0,0
1,2020-03-06,1,1,0,0,0,0
2,2020-03-07,1,0,0,0,0,0
3,2020-03-08,2,1,0,0,0,0
4,2020-03-09,3,1,0,0,0,0


In [7]:
#Number of actual active cases calculation
col_df['active'] = col_df.confirmed - col_df.deaths - col_df.recovered

## 1. Cases Evolution Over Time

Graphs for confirmed cases, deaths and recoveries over time are generated since the first case was confirmed (march 06). It is clear that it tends to have a exponential behavior.

First case corresponds to a 19 year old woman who arrived to the country the week before from Milan, Italy.

In [8]:
def plot_line(df):
    fig.add_shape(
       # Line Horizontal
          type="line",
            x0='2020-03-25',
            y0=0,
            x1='2020-03-25',
            y1=df.max(),
            line=dict(
                color="LightSeaGreen",
                width=4,
                
            ))
    fig.update_layout(
        showlegend=False,
        annotations=[
            dict(
                x='2020-03-25',
                y=df.max(),
                xref="x",
                yref="y",
                text="Quarantine Start",
                )
        ]
    )
    fig.show()

In [9]:
#Plotting
fig = px.line(col_df, x="date", y="confirmed", 
              title="Colombia Confirmed Cases")
plot_line(col_df.confirmed)


fig = px.line(col_df, x="date", y="deaths", 
              title="Colombia Confirmed Deaths")
plot_line(col_df.deaths)

fig = px.line(col_df, x="date", y="recovered", 
              title="Colombia Confirmed Recoveries")
plot_line(col_df.recovered)

Total of active cases

*Confirmed cases - Deaths - Recoveries*

In [10]:
fig = px.line(col_df, x="date", y="active", 
              title="Colombia Active Cases")
plot_line(col_df.active)

In [11]:
cols = confirmed.keys()
cols_d = deaths.keys()

confirmed1 = confirmed.loc[:, cols[1]:cols[-1]]
deaths1 = deaths.loc[:, cols_d[1]:cols_d[-1]]

In [12]:
#Number of days since the outbreak (March 6th)
days = np.array([i for i in range(len(col_df.index))]).reshape(-1, 1)

dates = confirmed1.keys()
state_cases = []
total_deaths = [] 

#Total number of cases
for i in dates:
    confirmed_sum = confirmed1[i].sum()
    #death_sum = deaths1[i].sum()
    
    state_cases.append(confirmed_sum)
    #total_deaths.append(death_sum)

print('Total number of confirmed cases: ',confirmed_sum)

Total number of confirmed cases:  1161


## 2. Number of Cases by State (Departamento)

Actual number of cases in each region. As of march 23, Bogota has most of the number of cases. Beign the capital, most of the international passengers traffic pass through it via El Dorado airport.

In [13]:
state_cases = np.array(state_cases).reshape(-1, 1)
#total_deaths = np.array(total_deaths).reshape(1, -1)

deptos = np.array(confirmed.state)
total = np.array(confirmed.loc[:,cols[-1]])
#total_d = np.array(deaths.loc[:,cols[-1]])

In [14]:
#Loading shapefile for the choropleth
deptos_geo = gpd.read_file('../input/colombia-shape-files-by-departments/depto.shp')
deptos_geo['NOMBRE_DPT'] = deptos_geo['NOMBRE_DPT'].str.capitalize()
deptos_geo.loc[2,'NOMBRE_DPT'] = 'Bogota'
deptos_geo.loc[32,'NOMBRE_DPT'] = 'San andres y providencia'
deptos_geo.loc[16,'NOMBRE_DPT'] = 'Narino'
deptos_geo = deptos_geo.sort_values(by=['NOMBRE_DPT'])
deptos_geo = deptos_geo[['NOMBRE_DPT','geometry']]
deptos_geo.set_index('NOMBRE_DPT', inplace=True)

In [15]:
#List of confirmed cases per state
df = pd.DataFrame({'NOMBRE_DPT':deptos,'confirmed':total})
df['NOMBRE_DPT'] = df['NOMBRE_DPT'].str.capitalize()
df = df.sort_values(by=['NOMBRE_DPT'])
df.set_index('NOMBRE_DPT', inplace=True)

In [16]:
df_merge = deptos_geo.merge(df,on='NOMBRE_DPT') 
df_merge

,geometry,confirmed
NOMBRE_DPT,,
Amazonas,"POLYGON ((1300074.955 504576.022, 1301222.910 ...",0
Antioquia,"POLYGON ((754872.146 1445316.567, 755882.980 1...",127
Arauca,"POLYGON ((1373898.278 1273429.779, 1374479.298...",0
Atlantico,"POLYGON ((913511.163 1637374.172, 912870.293 1...",35
Bogota,"POLYGON ((1006435.805 1021680.392, 1006048.264...",542
Bolivar,"POLYGON ((881889.901 1644370.282, 882454.535 1...",42
Boyaca,"POLYGON ((1206427.930 1268967.315, 1207191.680...",6
Caldas,"POLYGON ((931978.603 1127612.742, 933119.333 1...",16
Caqueta,"POLYGON ((931968.406 767660.187, 932356.113 76...",0


In [17]:
m_1 = folium.Map(location=[4,-73], tiles='cartodbpositron', zoom_start=5)

folium.Choropleth(geo_data=df_merge['geometry'],
           data=df_merge, columns=[df_merge.index, 'confirmed'],
           key_on="feature.id",
           fill_color='YlOrRd',
           legend_name='Number of confirmed cases'
           ).add_to(m_1)

embed_map(m_1, 'q_1.html')

In [18]:
#Plotting
df = pd.DataFrame({'state':deptos,'confirmed':total})
fig = px.bar(df.sort_values('confirmed', ascending=False)[:10][::-1], 
             x='confirmed', y='state', color_discrete_sequence=['#84DCC6'],
             title='Confirmed Cases by State', text='confirmed', orientation='h')
fig.show()

Now, a graph showing the number of deaths per state is generated.

The first death corresponds to a 58 year old taxi driver who passed away on march 16th in Cartagena after having contact with several international passengers by picking them up from the airport, but was officialy related to COVID-19 and reported until march 22nd.

In [19]:
total_deaths = []
total_deaths = cases.loc[cases['state'] == 'Fallecido']
total_deaths = total_deaths.groupby(['departamento']).count()
total_deaths = total_deaths.rename(columns={"ID": "n"}).drop(columns=['date','city','state','age', 'sex','type','procedence'])

total_deaths

,n
departamento,
Atlantico,1
Bogota D.C.,6
Cartagena D.T. y C,3
Cauca,1
Cundinamarca,1
Huila,1
Risaralda,1
Santa Marta D.T. y C.,1
Valle del Cauca,4


In [20]:
df = pd.DataFrame({'state':total_deaths.index,'confirmed':total_deaths.n})
fig = px.bar(df.sort_values('confirmed', ascending=False)[:10][::-1], 
             x='confirmed', y='state', color_discrete_sequence=['#84DCC6'],
             title='Confirmed Deaths by State', text='confirmed', orientation='h')
fig.show()

## 3. Mortality and Recovery Rate

Moratlity rate per total of confirmed cases. Initially, the World Health Organization made an estimate of 2% of deaths over the total number of cases globally.

A graph showing death rate over time in Colombia is generated

In [21]:
#Mortality and recovery rate calculation
col_df['death_rate'] = (col_df.deaths/col_df.confirmed) * 100
col_df['recover_rate'] = (col_df.recovered/col_df.confirmed) * 100
col_df['inf_rate'] = (col_df.confirmed/48258494) * 100

col_df

,date,confirmed,confirmed_daily,deaths,deaths_daily,recovered,recovered_daily,active,death_rate,recover_rate,inf_rate
0,2020-03-05,0,0,0,0,0,0,0,NaN,NaN,0.000000
1,2020-03-06,1,1,0,0,0,0,1,0.000000,0.000000,0.000002
2,2020-03-07,1,0,0,0,0,0,1,0.000000,0.000000,0.000002
3,2020-03-08,2,1,0,0,0,0,2,0.000000,0.000000,0.000004
4,2020-03-09,3,1,0,0,0,0,3,0.000000,0.000000,0.000006
5,2020-03-10,3,0,0,0,0,0,3,0.000000,0.000000,0.000006
6,2020-03-11,9,6,0,0,0,0,9,0.000000,0.000000,0.000019
7,2020-03-12,9,0,0,0,0,0,9,0.000000,0.000000,0.000019
8,2020-03-13,16,7,0,0,0,0,16,0.000000,0.000000,0.000033
9,2020-03-14,24,8,0,0,0,0,24,0.000000,0.000000,0.000050


In [22]:
#Temp dataframe for plotting multiple traces
df = pd.DataFrame([col_df.date,col_df.death_rate,col_df.recover_rate])
df_melt = col_df.melt(id_vars='date', value_vars=['death_rate', 'recover_rate'])

In [23]:
fig = px.line(df_melt, x="date", y="value", 
              title="Colombia Mortality and Recover Rate (%)",color='variable')

print('Death Rate: ',col_df.death_rate.max() , '%')
print('Recover Rate: ',col_df.recover_rate.max() , '%')
plot_line(df_melt.value)

Death Rate:  1.7660044150110374 %
Recover Rate:  4.7372954349698535 %


Infection rate over time assuming a population of 48'258.494 (result of the 2018 census)

In [24]:
fig = px.line(col_df, x="date", y="inf_rate", 
              title="Colombia Infection Rate (%) (Population: 48'258.494)")

print('Infecion Rate: ',col_df.inf_rate.max() , '%')
plot_line(col_df.inf_rate)

Infecion Rate:  0.002405794097097187 %


## 4. Confirmed Daily Cases

In [25]:
df = pd.DataFrame({'Date':col_df.date,'Confirmed':col_df.confirmed_daily})
fig = px.bar(df, y='Confirmed', x='Date', color_discrete_sequence=['#84DCC6'],
             title='Confirmed Daily Cases', text='Confirmed', orientation='v')
fig.show()

df = pd.DataFrame({'Date':col_df.date,'Deaths':col_df.deaths_daily})
fig = px.bar(df, y='Deaths', x='Date', color_discrete_sequence=['#84DCC6'],
             title='Confirmed Daily Deaths', text='Deaths', orientation='v')
fig.show()

df = pd.DataFrame({'Date':col_df.date,'Recovered':col_df.recovered_daily})
fig = px.bar(df, y='Recovered', x='Date', color_discrete_sequence=['#84DCC6'],
             title='Confirmed Daily Recoveries', text='Recovered', orientation='v')
fig.show()

In [26]:
df_melt = col_df.melt(id_vars='date', value_vars=['recovered_daily','deaths_daily', 'confirmed_daily'])
fig = px.bar(df_melt, y='value', x='date', color='variable',
             title='Confirmed Daily Cases', text='value', orientation='v',barmode='group')
fig.show()

## 5. Cases by Sex and Age Group

Number of total confirmed cases by sex and age groups

In [27]:
male = cases.loc[cases['sex'] == 'M'].count()[0]
female = cases.loc[cases['sex'] == 'F'].count()[0]

sex_grouped = pd.DataFrame({'M': [male], 'F': [female]}).T
sex_grouped.columns = ['n']
sex_grouped

,n
M,593
F,568


In [28]:
fig = px.pie(sex_grouped, values='n', names= sex_grouped.index,
             title='Cases by Sex')
fig.show()

Graph above shows that cases by sex are equally distributed, being males most ofthe infected cases.

Next plot shows that most of the infected cases are people in the 20-29 group, and the lowest number are in the 0-9 and 80-89 that are parte of the most vulnerable population.

In [29]:
age_grouped = cases.groupby(['age']).count()
age_grouped['ID']

age
0     1
1     1
2     5
3     2
4     1
     ..
83    3
84    6
85    2
87    2
88    2
Name: ID, Length: 85, dtype: int64

In [30]:
age_grouped

,ID,date,city,departamento,state,sex,type,procedence
age,,,,,,,,
0,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1
2,5,5,5,5,5,5,5,5
3,2,2,2,2,2,2,2,2
4,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
83,3,3,3,3,3,3,3,3
84,6,6,6,6,6,6,6,6
85,2,2,2,2,2,2,2,2


In [31]:
#fig = px.pie(age_grouped, values='ID', names= age_grouped.index,
#             title='Cases by Age Groups')
fig = px.bar(age_grouped, y='ID', x=age_grouped.index,
             title='Confirmed Daily Cases', text=age_grouped.index, orientation='v',barmode='group')
fig.show()

## 6. Actual Patents State

Actual state per confirmed case

- En casa (Recovering at home)
- En estudio (still a study case)
- Hospital
- Hospital UCI (patients in ICU)
- Fallecido (deaths)

In [32]:
state = cases.groupby(['state']).count()
state = state.rename(columns={"ID": "n"}).drop(columns=['date','city','departamento','age', 'sex','type','procedence'])

state

,n
state,
Casa,955
Fallecido,19
Hospital,82
Hospital UCI,50
Recuperado,55


In [33]:
fig = px.bar(state, y='n', x= state.index,
             title='Cases Actual State')
fig.show()

A graph showing number of patients over time in ICU (Intensive Care Unit). According to Registro Especial de Prestadores de Salud, as of march 2020, 5359 ICU bed units are available in the country. Also, according to Asociación de Medicina Crítica y Cuidado Intensivo (AMCI), 80% of this bed units are already ocuppied so it is urgent for the government to start looking for alternetives to increase the number of ICU units to treat future patients in critical conditions.

In [34]:
uci = cases.loc[cases['state'] == 'Hospital UCI'].groupby(['date','state']).count().reset_index()

In [35]:
dates = uci.date
total_uci = [uci.iloc[0,2]]

#total_uci.append(uci.iloc[0,2])
for i in range(len(dates)-1):
    uci_sum = total_uci[i] + uci.iloc[i+1,2]
    total_uci.append(uci_sum)
    
   

df = pd.DataFrame({'date':dates,'total':total_uci})

In [36]:
fig = px.line(df, x="date", y="total", 
              title="Actual Active Cases in ICU")
#fig.add_shape(
#       # Line Horizontal
#          type="line",
#            x0='2020-03-16',
#            y0=5359,
#            x1='2020-03-27',
#            y1=5359,
#            line=dict(
#                color="LightSeaGreen",
#                width=4,
#                dash="dashdot",
#            ))

plot_line(df.total)